## Outline of Notebook

At this point I have already:
- created a 3:2 sized image in MJ
- downloaded that image to 'downloads' folder locally, save as 'imagename_1x.png'

What this notebook will do:

MJ Image
- take image, rescale to 4:3 & 5:4 aspect ratios; and copy 1x image
- ensure two rescaled images and copied 1x image all have naming scheme 'imagename_AR.png' , be sure to replace AR with whatever aspect ratio it is being converted or copied to. Thus, the 1x image being copied will always be name 'imagename_32.png'
- UPSCAYL APP : run the download folder with 4 images through the pixel upscayl app
- run code to take images out of the folder within the path: /Users/Matt/Pictures/Midjourney/Imagine/Downloads , and replace them with the images already at this path
- run the DPI code on this path /Users/Matt/Pictures/Midjourney/Imagine/Downloads
- 4 images remain in the download folder

Mockup App Process

- 10 or so images should now be in the download folder after the mockups have been created
- all images should be named correctly
- create a folder within the prelisting folder based on the first name of the image 'imagename' in 'imagename_1x.png' from above
- move these images to that folder 
- within that 'imagename' folder, create two folders 'etsy' and 'google docs'
- sort the images with 'mockups' in their saved name and images with '1x' in their name to the etsy folder, the rest to the google docs folder

- Done

- download image from midjourney
- run first cell of code
- upscayl download folder
- run mockup app
- run second cell of code

------------

-----

## Landscape

### 1) Download and Rename 3:2 Image from Midjourney

### 2) Run the code below to create 4:3 and 5:4 variants

In [5]:
from PIL import Image
import os
import shutil

def analyze_image_name(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith("_1x.png"):
                return file, os.path.join(root, file)
    return None, None

def resize_image(image_path, aspect_ratio, new_name):
    with Image.open(image_path) as img:
        width, height = img.size
        
        # Calculate new dimensions based on aspect ratio
        if aspect_ratio == (4, 3):
            new_height = int(width * 3 / 4)
        elif aspect_ratio == (5, 4):
            new_height = int(width * 4 / 5)
        elif aspect_ratio == (14, 11):
            new_height = int(width * 11 / 14)
        elif aspect_ratio == (1, 1.414):  # ISO aspect ratio
            new_height = int(width / 1.414)
        else:
            raise ValueError("Unsupported aspect ratio")
        
        resized_img = img.resize((width, new_height), Image.LANCZOS)
        
        # Save the resized image
        resized_img.save(new_name)
        print(f"Saved resized image: {new_name}")

def process_images(directory):
    image_name, file_path = analyze_image_name(directory)
    
    if not image_name or not file_path:
        print("No _1x.png image found.")
        return
    
    base_name = image_name.split('_1x')[0]
    
    # Create new names for the resized and copied images
    new_name_43 = os.path.join(directory, f"{base_name}_43.png")
    new_name_54 = os.path.join(directory, f"{base_name}_54.png")
    new_name_11x14 = os.path.join(directory, f"{base_name}_14x11.png")
    new_name_ISO = os.path.join(directory, f"{base_name}_ISO.png")
    copied_name_32 = os.path.join(directory, f"{base_name}_32.png")
    
    # Resize and save the images
    resize_image(file_path, (4, 3), new_name_43)      # 4:3 ratio
    resize_image(file_path, (5, 4), new_name_54)      # 5:4 ratio
    resize_image(file_path, (14, 11), new_name_11x14) # 11x14 ratio
    resize_image(file_path, (1, 1.414), new_name_ISO) # ISO ratio (1:1.414)
    
    # Copy the original image to a new file with the _32 naming scheme
    shutil.copy(file_path, copied_name_32)
    print(f"Copied original image to: {copied_name_32}")

# Path to your download directory
download_directory = '/Users/matt/Pictures/Midjourney/Imagine/Downloads'

# Process images for resizing and copying
process_images(download_directory)


Saved resized image: /Users/matt/Pictures/Midjourney/Imagine/Downloads/fallporch_43.png
Saved resized image: /Users/matt/Pictures/Midjourney/Imagine/Downloads/fallporch_54.png
Saved resized image: /Users/matt/Pictures/Midjourney/Imagine/Downloads/fallporch_14x11.png
Saved resized image: /Users/matt/Pictures/Midjourney/Imagine/Downloads/fallporch_ISO.png
Copied original image to: /Users/matt/Pictures/Midjourney/Imagine/Downloads/fallporch_32.png


### 3) Upscayl the Downloads folder x8

### 4) Create Mockups with the mockup app

### 5) Run the code below to increase DPI to 300, and move images into prelisting folder

In [6]:
import os
import shutil
import subprocess
from PIL import Image

def move_and_replace_images_in_directory(directory):
    # Find the new folder within the download directory
    subfolders = [f.path for f in os.scandir(directory) if f.is_dir()]
    if not subfolders:
        print("No subfolder found in the downloads directory.")
        return

    new_folder = subfolders[0]  # Assuming there's only one subfolder
    print(f"Processing folder: {new_folder}")

    # Move images back to the parent directory and replace if names match
    for file_name in os.listdir(new_folder):
        source_path = os.path.join(new_folder, file_name)
        destination_path = os.path.join(directory, file_name)
        
        if os.path.exists(destination_path):
            os.remove(destination_path)
            print(f"Replaced: {destination_path}")
        
        shutil.move(source_path, directory)
        print(f"Moved: {source_path} to {directory}")
    
    # Safely delete the new folder, ensuring it's not named 'downloads'
    if os.path.basename(new_folder).lower() != 'downloads':
        shutil.rmtree(new_folder)
        print(f"Deleted folder: {new_folder}")
    else:
        print("Safety check: Attempted to delete a folder named 'downloads'. Aborting deletion.")

def set_dpi_exiftool(directory_path, dpi=300):
    try:
        # Construct the ExifTool command
        command = [
            "exiftool",
            "-overwrite_original", 
            "-r", 
            f"-XResolution={dpi}",
            f"-YResolution={dpi}",
            "-ResolutionUnit=2", 
            directory_path
        ]
        
        # Run the command using subprocess
        subprocess.run(command, check=True)
        print(f"Successfully set DPI to {dpi} for all images in {directory_path}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while setting DPI: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def adjust_dpi_for_selected_images(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if any(suffix in file for suffix in ["_43.png", "_54.png", "_32.png", "_ISO.png", "_14x11.png"]):
                file_path = os.path.join(root, file)
                set_dpi_exiftool(file_path, dpi=300)

def organize_images(directory, prelisting_folder):
    image_names = [file for file in os.listdir(directory) if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]
    
    # Determine base name from the first image with "_1x.png"
    base_name = None
    for image_name in image_names:
        if "_1x.png" in image_name:
            base_name = image_name.split('_1x')[0]
            break

    if not base_name:
        print("No _1x.png image found.")
        return
    
    # Create directories for organizing images
    new_dir = os.path.join(prelisting_folder, base_name)
    etsy_dir = os.path.join(new_dir, 'etsy')
    google_docs_dir = os.path.join(new_dir, 'google_docs')
    os.makedirs(etsy_dir, exist_ok=True)
    os.makedirs(google_docs_dir, exist_ok=True)
    
    for image_name in image_names:
        file_path = os.path.join(directory, image_name)
        
        if "_1x.png" in image_name or "mockup" in image_name.lower():
            shutil.move(file_path, os.path.join(etsy_dir, image_name))
            print(f"Moved {image_name} to {etsy_dir}")
        elif any(suffix in image_name for suffix in ["_43.png", "_54.png", "_32.png", "_ISO.png", "_14x11.png"]):
            shutil.move(file_path, os.path.join(google_docs_dir, image_name))
            print(f"Moved {image_name} to {google_docs_dir}")

    # Rename the files in the google_docs directory
    rename_files_in_google_docs(google_docs_dir)

def rename_files_in_google_docs(google_docs_dir):
    for file_name in os.listdir(google_docs_dir):
        # Initialize new_name as None
        new_name = None
        
        # Replace aspect ratio suffixes with appropriate format for horizontal/landscape variants
        if "_43.png" in file_name:
            new_name = "4x3.png"
        elif "_54.png" in file_name:
            new_name = "5x4.png"
        elif "_32.png" in file_name:
            new_name = "3x2.png"
        elif "_14x11.png" in file_name:
            new_name = "14x11.png"
        elif "_ISO.png" in file_name:
            new_name = "ISO.png"
        
        if new_name:
            old_path = os.path.join(google_docs_dir, file_name)
            new_path = os.path.join(google_docs_dir, new_name)
            os.rename(old_path, new_path)
            print(f"Renamed: {old_path} to {new_path}")

# Paths
download_directory = '/Users/matt/Pictures/Midjourney/Imagine/Downloads'
prelisting_folder = '/Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Matisse/Prelisting'

# Move and replace images, then delete the subfolder
move_and_replace_images_in_directory(download_directory)

# Adjust DPI for selected images
adjust_dpi_for_selected_images(download_directory)

# Organize images into folders and rename them in google_docs folder
organize_images(download_directory, prelisting_folder)


Processing folder: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x4plus_6x
Replaced: /Users/matt/Pictures/Midjourney/Imagine/Downloads/fallporch_43.png
Moved: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x4plus_6x/fallporch_43.png to /Users/matt/Pictures/Midjourney/Imagine/Downloads
Replaced: /Users/matt/Pictures/Midjourney/Imagine/Downloads/fallporch_14x11.png
Moved: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x4plus_6x/fallporch_14x11.png to /Users/matt/Pictures/Midjourney/Imagine/Downloads
Replaced: /Users/matt/Pictures/Midjourney/Imagine/Downloads/fallporch_54.png
Moved: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x4plus_6x/fallporch_54.png to /Users/matt/Pictures/Midjourney/Imagine/Downloads
Replaced: /Users/matt/Pictures/Midjourney/Imagine/Downloads/fallporch_32.png
Moved: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x4plus_6x/fallporch_32.p

    1 image files updated
Successfully set DPI to 300 for all images in /Users/matt/Pictures/Midjourney/Imagine/Downloads/fallporch_14x11.png
    1 image files updated
Successfully set DPI to 300 for all images in /Users/matt/Pictures/Midjourney/Imagine/Downloads/fallporch_54.png


    1 image files updated
Successfully set DPI to 300 for all images in /Users/matt/Pictures/Midjourney/Imagine/Downloads/fallporch_32.png
    1 image files updated
Successfully set DPI to 300 for all images in /Users/matt/Pictures/Midjourney/Imagine/Downloads/fallporch_ISO.png
Moved fallporch_mockup_0021.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Matisse/Prelisting/fallporch/etsy
Moved fallporch_43.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Matisse/Prelisting/fallporch/google_docs
Moved fallporch_14x11.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Matisse/Prelisting/fallporch/google_docs
Moved fallporch_54.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Matisse/Prelisting/fallporch/google_docs
Moved fallporch_32.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Matisse/Prelisting/fallporch/google_docs
Moved fallporch_mockup_0055.png to /Users/matt/Pictures/Midjourney/Digi

----

## Portrait

### 1) Download and Rename 2:3 Image from Midjourney

### 2) Run the code below to create 3:4 and 4:5 variants

In [10]:
from PIL import Image
import os
import shutil

def analyze_image_name(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith("_1x.png"):
                return file, os.path.join(root, file)
    return None, None

def resize_image(image_path, aspect_ratio, new_name):
    with Image.open(image_path) as img:
        width, height = img.size
        
        # Calculate new dimensions
        if aspect_ratio == (3, 4):  # 3:4 aspect ratio (vertical)
            new_height = int(width * 4 / 3)
            resized_img = img.resize((width, new_height), Image.LANCZOS)
        elif aspect_ratio == (4, 5):  # 4:5 aspect ratio (vertical)
            new_height = int(width * 5 / 4)
            resized_img = img.resize((width, new_height), Image.LANCZOS)
        elif aspect_ratio == (11, 14):  # 11x14 aspect ratio (vertical)
            new_height = int(width * 14 / 11)
            resized_img = img.resize((width, new_height), Image.LANCZOS)
        elif aspect_ratio == (1, 1.414):  # ISO aspect ratio (vertical)
            new_height = int(width * 1.414)
            resized_img = img.resize((width, new_height), Image.LANCZOS)
        else:
            raise ValueError("Unsupported aspect ratio")
        
        # Save the resized image
        resized_img.save(new_name)
        print(f"Saved resized image: {new_name}")

def process_images(directory):
    image_name, file_path = analyze_image_name(directory)
    
    if not image_name or not file_path:
        print("No _1x.png image found.")
        return
    
    base_name = image_name.split('_1x')[0]
    
    # Create new names for the resized and copied images
    new_name_34 = os.path.join(directory, f"{base_name}_34.png")
    new_name_45 = os.path.join(directory, f"{base_name}_45.png")
    new_name_14x11 = os.path.join(directory, f"{base_name}_11x14.png")
    new_name_ISO = os.path.join(directory, f"{base_name}_ISO.png")
    copied_name_23 = os.path.join(directory, f"{base_name}_23.png")
    
    # Resize and save the images
    resize_image(file_path, (3, 4), new_name_34)      # 3:4 ratio (vertical)
    resize_image(file_path, (4, 5), new_name_45)      # 4:5 ratio (vertical)
    resize_image(file_path, (11, 14), new_name_14x11) # 14x11 ratio (vertical)
    resize_image(file_path, (1, 1.414), new_name_ISO) # ISO ratio (1:1.414, vertical)
    
    # Copy the original image to a new file with the _23 naming scheme
    shutil.copy(file_path, copied_name_23)
    print(f"Copied original image to: {copied_name_23}")

# Path to your download directory
download_directory = '/Users/matt/Pictures/Midjourney/Imagine/Downloads'

# Process images for resizing and copying
process_images(download_directory)


Saved resized image: /Users/matt/Pictures/Midjourney/Imagine/Downloads/pumpkinporch_34.png
Saved resized image: /Users/matt/Pictures/Midjourney/Imagine/Downloads/pumpkinporch_45.png
Saved resized image: /Users/matt/Pictures/Midjourney/Imagine/Downloads/pumpkinporch_11x14.png
Saved resized image: /Users/matt/Pictures/Midjourney/Imagine/Downloads/pumpkinporch_ISO.png
Copied original image to: /Users/matt/Pictures/Midjourney/Imagine/Downloads/pumpkinporch_23.png


### 3) Upscayl the Downloads folder

### 4) Create Mockups with the mockup app

### 5) Run the code below to increase DPI to 300, and move images into prelisting folder

In [11]:
import os
import shutil
import subprocess
from PIL import Image

def move_and_replace_images_in_directory(directory):
    # Find the new folder within the download directory
    subfolders = [f.path for f in os.scandir(directory) if f.is_dir()]
    if not subfolders:
        print("No subfolder found in the downloads directory.")
        return

    new_folder = subfolders[0]  # Assuming there's only one subfolder
    print(f"Processing folder: {new_folder}")

    # Move images back to the parent directory and replace if names match
    for file_name in os.listdir(new_folder):
        source_path = os.path.join(new_folder, file_name)
        destination_path = os.path.join(directory, file_name)
        
        if os.path.exists(destination_path):
            os.remove(destination_path)
            print(f"Replaced: {destination_path}")
        
        shutil.move(source_path, directory)
        print(f"Moved: {source_path} to {directory}")
    
    # Safely delete the new folder, ensuring it's not named 'downloads'
    if os.path.basename(new_folder).lower() != 'downloads':
        shutil.rmtree(new_folder)
        print(f"Deleted folder: {new_folder}")
    else:
        print("Safety check: Attempted to delete a folder named 'downloads'. Aborting deletion.")

def set_dpi_exiftool(directory_path, dpi=300):
    try:
        # Construct the ExifTool command
        command = [
            "exiftool",
            "-overwrite_original", 
            "-r", 
            f"-XResolution={dpi}",
            f"-YResolution={dpi}",
            "-ResolutionUnit=2", 
            directory_path
        ]
        
        # Run the command using subprocess
        subprocess.run(command, check=True)
        print(f"Successfully set DPI to {dpi} for all images in {directory_path}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while setting DPI: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def adjust_dpi_for_selected_images(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if any(suffix in file for suffix in ["_34.png", "_45.png", "_23.png", "_ISO.png", "_11x14.png"]):
                file_path = os.path.join(root, file)
                set_dpi_exiftool(file_path, dpi=300)

def organize_images(directory, prelisting_folder):
    image_names = [file for file in os.listdir(directory) if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]
    
    # Determine base name from the first image with "_1x.png"
    base_name = None
    for image_name in image_names:
        if "_1x.png" in image_name:
            base_name = image_name.split('_1x')[0]
            break

    if not base_name:
        print("No _1x.png image found.")
        return
    
    # Create directories for organizing images
    new_dir = os.path.join(prelisting_folder, base_name)
    etsy_dir = os.path.join(new_dir, 'etsy')
    google_docs_dir = os.path.join(new_dir, 'google_docs')
    os.makedirs(etsy_dir, exist_ok=True)
    os.makedirs(google_docs_dir, exist_ok=True)
    
    for image_name in image_names:
        file_path = os.path.join(directory, image_name)
        
        if "_1x.png" in image_name or "mockup" in image_name.lower():
            shutil.move(file_path, os.path.join(etsy_dir, image_name))
            print(f"Moved {image_name} to {etsy_dir}")
        elif any(suffix in image_name for suffix in ["_34.png", "_45.png", "_23.png", "_ISO.png", "_11x14.png"]):
            shutil.move(file_path, os.path.join(google_docs_dir, image_name))
            print(f"Moved {image_name} to {google_docs_dir}")

    # Rename the files in the google_docs directory
    rename_files_in_google_docs(google_docs_dir)

def rename_files_in_google_docs(google_docs_dir):
    for file_name in os.listdir(google_docs_dir):
        # Initialize new_name as None
        new_name = None
        
        # Replace aspect ratio suffixes with appropriate format for vertical variants
        if "_34.png" in file_name:
            new_name = "3x4.png"
        elif "_45.png" in file_name:
            new_name = "4x5.png"
        elif "_23.png" in file_name:
            new_name = "2x3.png"
        elif "_11x14.png" in file_name:
            new_name = "11x14.png"
        elif "_ISO.png" in file_name:
            new_name = "ISO.png"
        
        if new_name:
            old_path = os.path.join(google_docs_dir, file_name)
            new_path = os.path.join(google_docs_dir, new_name)
            os.rename(old_path, new_path)
            print(f"Renamed: {old_path} to {new_path}")

# Paths
download_directory = '/Users/matt/Pictures/Midjourney/Imagine/Downloads'
prelisting_folder = '/Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Matisse/Prelisting'

# Move and replace images, then delete the subfolder
move_and_replace_images_in_directory(download_directory)

# Adjust DPI for selected images
adjust_dpi_for_selected_images(download_directory)

# Organize images into folders and rename them in google_docs folder
organize_images(download_directory, prelisting_folder)


Processing folder: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x4plus_6x
Replaced: /Users/matt/Pictures/Midjourney/Imagine/Downloads/pumpkinporch_ISO.png
Moved: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x4plus_6x/pumpkinporch_ISO.png to /Users/matt/Pictures/Midjourney/Imagine/Downloads
Replaced: /Users/matt/Pictures/Midjourney/Imagine/Downloads/pumpkinporch_45.png
Moved: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x4plus_6x/pumpkinporch_45.png to /Users/matt/Pictures/Midjourney/Imagine/Downloads
Replaced: /Users/matt/Pictures/Midjourney/Imagine/Downloads/pumpkinporch_11x14.png
Moved: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x4plus_6x/pumpkinporch_11x14.png to /Users/matt/Pictures/Midjourney/Imagine/Downloads
Replaced: /Users/matt/Pictures/Midjourney/Imagine/Downloads/pumpkinporch_34.png
Moved: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x

    1 image files updated
Successfully set DPI to 300 for all images in /Users/matt/Pictures/Midjourney/Imagine/Downloads/pumpkinporch_45.png
    1 image files updated
Successfully set DPI to 300 for all images in /Users/matt/Pictures/Midjourney/Imagine/Downloads/pumpkinporch_11x14.png


    1 image files updated
Successfully set DPI to 300 for all images in /Users/matt/Pictures/Midjourney/Imagine/Downloads/pumpkinporch_34.png
    1 image files updated
Successfully set DPI to 300 for all images in /Users/matt/Pictures/Midjourney/Imagine/Downloads/pumpkinporch_23.png
Moved pumpkinporch_ISO.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Matisse/Prelisting/pumpkinporch/google_docs
Moved pumpkinporch_mockup_0043.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Matisse/Prelisting/pumpkinporch/etsy
Moved pumpkinporch_mockup_0019.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Matisse/Prelisting/pumpkinporch/etsy
Moved pumpkinporch_mockup_0031.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Matisse/Prelisting/pumpkinporch/etsy
Moved pumpkinporch_mockup_0008.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Matisse/Prelisting/pumpkinporch/etsy
Moved pumpkinporch_mockup_0013.p